In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import json

# List of JSON files to merge
json_files = [
    "/content/drive/Shareddrives/Kickstar_team3/Chelsie_building_video/Victory_Salute1.json",
    "/content/drive/Shareddrives/Kickstar_team3/Connor_Building_Video/Ceasar_Chavez_Memorial1.json",

    "/content/drive/Shareddrives/Kickstar_team3/Joon_building_video/CVB1_final.json",
    "/content/drive/Shareddrives/Kickstar_team3/Joon_building_video/CVB2_final.json",
    "/content/drive/Shareddrives/Kickstar_team3/Joon_building_video/CVB5_final.json",

    "/content/drive/Shareddrives/Kickstar_team3/Connor_Building_Video/Macquarrie_hall_1_final.json",
    "/content/drive/Shareddrives/Kickstar_team3/Connor_Building_Video/Macquarrie_hall_3_final.json"
]

merged_data = {"annotations": []}  # Initialize merged structure

# Read and merge data from all JSON files
for file in json_files:
    with open(file, "r") as f:
        data = json.load(f)
        merged_data["annotations"].extend(data["annotations"])  # Combine annotations

output_file = "/content/drive/Shareddrives/Kickstar_team3/merged_annotations.json"
with open(output_file, "w") as f:
    json.dump(merged_data, f, indent=4)

print(f"Merging completed. Merged file saved at: {output_file}")


Merging completed. Merged file saved at: /content/drive/Shareddrives/Kickstar_team3/merged_annotations.json


In [ ]:
import json
import os

# Load your JSON data
with open('/content/drive/Shareddrives/Kickstar_team3/merged_annotations.json') as f:
    data = json.load(f)

# Function to extract filename from the image path
def extract_filename(image_path):
    return os.path.basename(image_path)

# Create a new dictionary to store the new structure
new_data = {"annotations": []}

# Loop through the annotations, extract filenames, and create the new structure
for annotation in data['annotations']:
    image_path = annotation['image']
    filename = extract_filename(image_path)

    # Create a new annotation entry with just the filename
    new_annotation = {
        "image": filename,  # Store only the filename
        "instances": annotation['instances']  # Keep the instances as they are
    }

    # Add the new annotation to the new_data dictionary
    new_data["annotations"].append(new_annotation)

# Write the new data to a new JSON file
with open('/content/drive/Shareddrives/Kickstar_team3/processed_annotations.json', 'w') as outfile:
    json.dump(new_data, outfile, indent=4)

print("New JSON file with filenames created: processed_annotations.json")



New JSON file with filenames created: processed_annotations.json


In [ ]:

source_dirs = [
    '/content/drive/Shareddrives/Kickstar_team3/Connor_Building_Video/frame_memorial',
    '/content/drive/Shareddrives/Kickstar_team3/Chelsie_building_video/frame'
]

# Destination directory
destination_dir = '/content/drive/Shareddrives/Kickstar_team3/images'

# Ensure the destination directory exists, if not create it
if not os.path.exists(destination_dir):
    os.makedirs(destination_dir)

# Loop through the source directories
for source_dir in source_dirs:
    # Loop through each file in the source directory
    for filename in os.listdir(source_dir):
        # Check if the file is a .jpg image
        if filename.endswith('.jpg'):
            # Construct full file path
            source_file = os.path.join(source_dir, filename)
            destination_file = os.path.join(destination_dir, filename)

            # Copy the file to the destination directory
            shutil.copy(source_file, destination_file)
            print(f'Copied: {filename}')

print('All images copied successfully!')


Streaming output truncated to the last 5000 lines.
Copied: Ceasar_Chavez_Memorial_poisson_2721.jpg
Copied: Ceasar_Chavez_Memorial_2727.jpg
Copied: Ceasar_Chavez_Memorial_gaussian_2727.jpg
Copied: Ceasar_Chavez_Memorial_salt_pepper_2727.jpg
Copied: Ceasar_Chavez_Memorial_poisson_2727.jpg
Copied: Ceasar_Chavez_Memorial_2733.jpg
Copied: Ceasar_Chavez_Memorial_gaussian_2733.jpg
Copied: Ceasar_Chavez_Memorial_salt_pepper_2733.jpg
Copied: Ceasar_Chavez_Memorial_poisson_2733.jpg
Copied: Ceasar_Chavez_Memorial_2739.jpg
Copied: Ceasar_Chavez_Memorial_gaussian_2739.jpg
Copied: Ceasar_Chavez_Memorial_salt_pepper_2739.jpg
Copied: Ceasar_Chavez_Memorial_poisson_2739.jpg
Copied: Ceasar_Chavez_Memorial_2745.jpg
Copied: Ceasar_Chavez_Memorial_gaussian_2745.jpg
Copied: Ceasar_Chavez_Memorial_salt_pepper_2745.jpg
Copied: Ceasar_Chavez_Memorial_poisson_2745.jpg
Copied: Ceasar_Chavez_Memorial_2751.jpg
Copied: Ceasar_Chavez_Memorial_gaussian_2751.jpg
Copied: Ceasar_Chavez_Memorial_salt_pepper_2751.jpg
Copi

In [ ]:
import json
import os

# Your class names
class_names = ["Ceasar_Chavez_Memorial", "Macquarrie_hall", "Victory_Salute", "CVB"]

# Create the directory if it doesn't exist
output_dir = '/content/drive/Shareddrives/Kickstar_team3/yolotxt'
os.makedirs(output_dir, exist_ok=True)

# Load your JSON data
with open('/content/drive/Shareddrives/Kickstar_team3/merged_annotations.json') as f:
    data = json.load(f)

# Function to convert bbox to YOLO format (normalized)
def convert_bbox_to_yolo(bbox, img_width, img_height):
    x_min, y_min, width, height = bbox
    x_center = (x_min + width / 2) / img_width
    y_center = (y_min + height / 2) / img_height
    norm_width = width / img_width
    norm_height = height / img_height
    return [x_center, y_center, norm_width, norm_height]

# Loop through each annotation and write to text files
for annotation in data['annotations']:
    image_path = annotation['image']
    instances = annotation['instances']

    # Get the image name without extension
    image_name = os.path.splitext(os.path.basename(image_path))[0]

    # Get the image width and height (adjust this to your image dimensions)
    # For now, using placeholders: you need actual dimensions for proper normalization.
    img_width = 640  # Replace with actual image width if known
    img_height = 640  # Replace with actual image height if known

    # Prepare list to store bbox data
    bboxes = []

    for instance in instances:
        bbox = instance['bbox']
        label = instance['bbox_label']

        # Get the class ID
        class_id = class_names.index(label)

        # Convert to YOLO format
        yolo_bbox = convert_bbox_to_yolo(bbox, img_width, img_height)

        # Append the class ID and bbox to the list
        bboxes.append([class_id] + yolo_bbox)

    # Write the YOLO format data to a text file
    if bboxes:  # Only write if there are bounding boxes
        txt_file_path = os.path.join(output_dir, f"{image_name}.txt")
        with open(txt_file_path, 'w') as txt_file:
            for bbox in bboxes:
                txt_file.write(" ".join(map(str, bbox)) + "\n")

print("Conversion completed and files saved.")


Conversion completed and files saved.


In [ ]:
import os
import random
import shutil

# Path to your images directory
image_dir = '/content/drive/Shareddrives/Kickstar_team3/images/'

# Output directories for train and val
train_dir = '/content/drive/Shareddrives/Kickstar_team3/images/train'
val_dir = '/content/drive/Shareddrives/Kickstar_team3/images/val'

# Create directories if they do not exist
os.makedirs(train_dir, exist_ok=True)
os.makedirs(val_dir, exist_ok=True)

# List all the image files in the source directory
image_files = [f for f in os.listdir(image_dir) if f.endswith('.jpg')]

# Shuffle the image list to ensure randomness
random.shuffle(image_files)

# Split into train (90%) and val (10%)
split_index = int(0.9 * len(image_files))
train_files = image_files[:split_index]
val_files = image_files[split_index:]

# Function to move images to the target directory
def move_images(images, target_dir):
    for image in images:
        # Construct full source and target file paths
        source_path = os.path.join(image_dir, image)
        target_path = os.path.join(target_dir, image)

        # Move the image to the target directory
        shutil.move(source_path, target_path)
        print(f'Moved: {image} to {target_dir}')

# Move train and val images
move_images(train_files, train_dir)
move_images(val_files, val_dir)

print('Image splitting and moving completed!')


Streaming output truncated to the last 5000 lines.
Moved: Ceasar_Chavez_Memorial_salt_pepper_1128.jpg to /content/drive/Shareddrives/Kickstar_team3/images/train
Moved: Ceasar_Chavez_Memorial_0480.jpg to /content/drive/Shareddrives/Kickstar_team3/images/train
Moved: CVB_poisson_0327_frame2.jpg to /content/drive/Shareddrives/Kickstar_team3/images/train
Moved: Victory_Salute_poisson_1839.jpg to /content/drive/Shareddrives/Kickstar_team3/images/train
Moved: Macquarrie_hall_poisson_1116_frame1.jpg to /content/drive/Shareddrives/Kickstar_team3/images/train
Moved: Macquarrie_hall_1344_frame1.jpg to /content/drive/Shareddrives/Kickstar_team3/images/train
Moved: Ceasar_Chavez_Memorial_2277.jpg to /content/drive/Shareddrives/Kickstar_team3/images/train
Moved: Ceasar_Chavez_Memorial_gaussian_1290.jpg to /content/drive/Shareddrives/Kickstar_team3/images/train
Moved: Ceasar_Chavez_Memorial_gaussian_3249.jpg to /content/drive/Shareddrives/Kickstar_team3/images/train
Moved: Macquarrie_hall_salt_peppe

In [ ]:
import os
import shutil
import logging

# Update these paths to your own directory structure
train_image_dir = '/content/drive/Shareddrives/Kickstar_team3/dataset/images/train'  # Train images folder
val_image_dir = '/content/drive/Shareddrives/Kickstar_team3/dataset/images/val'      # Validation images folder
label_dir = '/content/drive/Shareddrives/Kickstar_team3/dataset/label/'              # All label files folder

# Paths where labels will be moved
train_label_dir = '/content/drive/Shareddrives/Kickstar_team3/dataset/label/train/'
val_label_dir = '/content/drive/Shareddrives/Kickstar_team3/dataset/label/val/'

# Create the train and val label directories if they don't exist
os.makedirs(train_label_dir, exist_ok=True)
os.makedirs(val_label_dir, exist_ok=True)

# Set up logging for tracking
logging.basicConfig(filename='label_move_log.txt', level=logging.INFO, format='%(asctime)s - %(message)s')

# Function to move label files to the correct directory based on the image folder
def move_labels(image_dir, label_dir, target_label_dir):
    for image_file in os.listdir(image_dir):
        if image_file.endswith('.jpg') or image_file.endswith('.png'):
            # Get the image name without the extension
            image_name = os.path.splitext(image_file)[0]
            label_file = image_name + '.txt'  # Corresponding label file

            # Check if the label file exists in the label directory
            label_path = os.path.join(label_dir, label_file)
            if os.path.exists(label_path):
                try:
                    # Move the label file to the correct target directory (train or val)
                    shutil.move(label_path, os.path.join(target_label_dir, label_file))
                    logging.info(f"Moved {label_file} to {target_label_dir}")  # Log the move
                    print(f"Moved {label_file} to {target_label_dir}")  # Print message for real-time feedback
                except Exception as e:
                    logging.error(f"Error moving {label_file}: {str(e)}")
                    print(f"Error moving {label_file}: {str(e)}")  # Print any errors
            else:
                logging.warning(f"Label file for {image_file} not found.")
                print(f"Label file for {image_file} not found.")  # Warn if no corresponding label file exists

# Move labels for the train set
move_labels(train_image_dir, label_dir, train_label_dir)

# Move labels for the val set
move_labels(val_image_dir, label_dir, val_label_dir)

logging.info("Label files have been successfully moved to the train and val folders.")
print("Label files have been successfully moved to the train and val folders.")


Moved Macquarrie_hall_gaussian_1620_frame3.txt to /content/drive/Shareddrives/Kickstar_team3/dataset/label/train/
Moved Macquarrie_hall_1608_frame3.txt to /content/drive/Shareddrives/Kickstar_team3/dataset/label/train/
Moved Macquarrie_hall_gaussian_1632_frame3.txt to /content/drive/Shareddrives/Kickstar_team3/dataset/label/train/
Moved Macquarrie_hall_salt_pepper_1632_frame3.txt to /content/drive/Shareddrives/Kickstar_team3/dataset/label/train/
Moved Macquarrie_hall_poisson_1632_frame3.txt to /content/drive/Shareddrives/Kickstar_team3/dataset/label/train/
Moved Macquarrie_hall_gaussian_1644_frame3.txt to /content/drive/Shareddrives/Kickstar_team3/dataset/label/train/
Moved Macquarrie_hall_salt_pepper_1644_frame3.txt to /content/drive/Shareddrives/Kickstar_team3/dataset/label/train/
Moved Macquarrie_hall_1656_frame3.txt to /content/drive/Shareddrives/Kickstar_team3/dataset/label/train/
Moved Macquarrie_hall_1644_frame3.txt to /content/drive/Shareddrives/Kickstar_team3/dataset/label/tra

Moved Macquarrie_hall_salt_pepper_4680_frame3.txt to /content/drive/Shareddrives/Kickstar_team3/dataset/label/train/
Moved Macquarrie_hall_gaussian_4680_frame3.txt to /content/drive/Shareddrives/Kickstar_team3/dataset/label/train/
Moved Macquarrie_hall_4692_frame3.txt to /content/drive/Shareddrives/Kickstar_team3/dataset/label/train/
Moved Macquarrie_hall_poisson_4680_frame3.txt to /content/drive/Shareddrives/Kickstar_team3/dataset/label/train/
Moved Macquarrie_hall_poisson_4692_frame3.txt to /content/drive/Shareddrives/Kickstar_team3/dataset/label/train/
Moved Macquarrie_hall_4704_frame3.txt to /content/drive/Shareddrives/Kickstar_team3/dataset/label/train/
Moved Macquarrie_hall_salt_pepper_4704_frame3.txt to /content/drive/Shareddrives/Kickstar_team3/dataset/label/train/
Moved Macquarrie_hall_gaussian_4704_frame3.txt to /content/drive/Shareddrives/Kickstar_team3/dataset/label/train/
Moved Macquarrie_hall_4716_frame3.txt to /content/drive/Shareddrives/Kickstar_team3/dataset/label/trai

Label file for Ceasar_Chavez_Memorial_2892_train.jpg not found.
Label file for CVB_0064_frame1_train.jpg not found.
Label file for Ceasar_Chavez_Memorial_gaussian_0933_train.jpg not found.
Label file for Ceasar_Chavez_Memorial_gaussian_2265_train.jpg not found.
Label file for Macquarrie_hall_gaussian_0012_frame1_train.jpg not found.
Label file for Victory_Salute_gaussian_0063_train.jpg not found.
Label file for Macquarrie_hall_2244_frame1_train.jpg not found.
Label file for Macquarrie_hall_poisson_0996_frame3_train.jpg not found.
Label file for Victory_Salute_gaussian_0954_train.jpg not found.
Label file for Ceasar_Chavez_Memorial_gaussian_0963_train.jpg not found.
Label file for Ceasar_Chavez_Memorial_poisson_3273_train.jpg not found.
Label file for Macquarrie_hall_gaussian_4080_frame3_train.jpg not found.
Label file for Ceasar_Chavez_Memorial_gaussian_2640_train.jpg not found.
Label file for Ceasar_Chavez_Memorial_1884_train.jpg not found.
Label file for Ceasar_Chavez_Memorial_1296_t

Moved Victory_Salute_poisson_0546.txt to /content/drive/Shareddrives/Kickstar_team3/dataset/label/train/
Moved Victory_Salute_0549.txt to /content/drive/Shareddrives/Kickstar_team3/dataset/label/train/
Moved Victory_Salute_gaussian_0549.txt to /content/drive/Shareddrives/Kickstar_team3/dataset/label/train/
Moved Victory_Salute_salt_pepper_0549.txt to /content/drive/Shareddrives/Kickstar_team3/dataset/label/train/
Moved Victory_Salute_poisson_0549.txt to /content/drive/Shareddrives/Kickstar_team3/dataset/label/train/
Moved Victory_Salute_0552.txt to /content/drive/Shareddrives/Kickstar_team3/dataset/label/train/
Moved Victory_Salute_gaussian_0552.txt to /content/drive/Shareddrives/Kickstar_team3/dataset/label/train/
Moved Victory_Salute_salt_pepper_0552.txt to /content/drive/Shareddrives/Kickstar_team3/dataset/label/train/
Moved Victory_Salute_poisson_0552.txt to /content/drive/Shareddrives/Kickstar_team3/dataset/label/train/
Moved Victory_Salute_0555.txt to /content/drive/Shareddrives/

Moved Victory_Salute_poisson_0576.txt to /content/drive/Shareddrives/Kickstar_team3/dataset/label/train/
Moved Victory_Salute_0579.txt to /content/drive/Shareddrives/Kickstar_team3/dataset/label/train/
Moved Victory_Salute_gaussian_0579.txt to /content/drive/Shareddrives/Kickstar_team3/dataset/label/train/
Moved Victory_Salute_salt_pepper_0579.txt to /content/drive/Shareddrives/Kickstar_team3/dataset/label/train/
Moved Victory_Salute_poisson_0579.txt to /content/drive/Shareddrives/Kickstar_team3/dataset/label/train/
Moved Victory_Salute_0582.txt to /content/drive/Shareddrives/Kickstar_team3/dataset/label/train/
Moved Victory_Salute_gaussian_0582.txt to /content/drive/Shareddrives/Kickstar_team3/dataset/label/train/
Label file for Victory_Salute_salt_pepper_0582.jpg not found.
Moved Victory_Salute_poisson_0582.txt to /content/drive/Shareddrives/Kickstar_team3/dataset/label/train/
Moved Victory_Salute_0585.txt to /content/drive/Shareddrives/Kickstar_team3/dataset/label/train/
Moved Victo

...

Label file for Ceasar_Chavez_Memorial_poisson_2559.jpg not found.
Label file for Ceasar_Chavez_Memorial_salt_pepper_2577.jpg not found.
Label file for Ceasar_Chavez_Memorial_2583.jpg not found.
Label file for Ceasar_Chavez_Memorial_gaussian_2583.jpg not found.
Label file for Ceasar_Chavez_Memorial_gaussian_2589.jpg not found.
Label file for Ceasar_Chavez_Memorial_2595.jpg not found.
Label file for Ceasar_Chavez_Memorial_gaussian_2643.jpg not found.
Label file for Ceasar_Chavez_Memorial_salt_pepper_2649.jpg not found.
Label file for Ceasar_Chavez_Memorial_poisson_2649.jpg not found.
Label file for Ceasar_Chavez_Memorial_poisson_2667.jpg not found.
Label file for Ceasar_Chavez_Memorial_0411.jpg not found.
Label file for Ceasar_Chavez_Memorial_salt_pepper_0411.jpg not found.
Label file for Ceasar_Chavez_Memorial_salt_pepper_0423.jpg not found.
Label file for Ceasar_Chavez_Memorial_gaussian_0429.jpg not found.
Label file for Ceasar_Chavez_Memorial_salt_pepper_0429.jpg not found.
Label file

Moved Victory_Salute_salt_pepper_1242.txt to /content/drive/Shareddrives/Kickstar_team3/dataset/label/val/
Moved Victory_Salute_salt_pepper_1251.txt to /content/drive/Shareddrives/Kickstar_team3/dataset/label/val/
Moved Victory_Salute_1257.txt to /content/drive/Shareddrives/Kickstar_team3/dataset/label/val/
Moved Victory_Salute_salt_pepper_1260.txt to /content/drive/Shareddrives/Kickstar_team3/dataset/label/val/
Moved Victory_Salute_gaussian_1272.txt to /content/drive/Shareddrives/Kickstar_team3/dataset/label/val/
Moved Victory_Salute_1278.txt to /content/drive/Shareddrives/Kickstar_team3/dataset/label/val/
Moved Victory_Salute_poisson_1278.txt to /content/drive/Shareddrives/Kickstar_team3/dataset/label/val/
Moved Victory_Salute_salt_pepper_1299.txt to /content/drive/Shareddrives/Kickstar_team3/dataset/label/val/
Moved Victory_Salute_gaussian_1314.txt to /content/drive/Shareddrives/Kickstar_team3/dataset/label/val/
Moved Victory_Salute_salt_pepper_1314.txt to /content/drive/Shareddrive

Moved Macquarrie_hall_gaussian_4620_frame3.txt to /content/drive/Shareddrives/Kickstar_team3/dataset/label/val/
Moved Macquarrie_hall_salt_pepper_4692_frame3.txt to /content/drive/Shareddrives/Kickstar_team3/dataset/label/val/
Moved Macquarrie_hall_gaussian_4692_frame3.txt to /content/drive/Shareddrives/Kickstar_team3/dataset/label/val/
Moved Macquarrie_hall_salt_pepper_4716_frame3.txt to /content/drive/Shareddrives/Kickstar_team3/dataset/label/val/
Moved CVB_gaussian_0021_frame2.txt to /content/drive/Shareddrives/Kickstar_team3/dataset/label/val/
Moved CVB_salt_pepper_0021_frame2.txt to /content/drive/Shareddrives/Kickstar_team3/dataset/label/val/
Moved CVB_0039_frame2.txt to /content/drive/Shareddrives/Kickstar_team3/dataset/label/val/
Moved CVB_gaussian_0039_frame2.txt to /content/drive/Shareddrives/Kickstar_team3/dataset/label/val/
Moved CVB_gaussian_0060_frame2.txt to /content/drive/Shareddrives/Kickstar_team3/dataset/label/val/
Moved CVB_poisson_0066_frame2.txt to /content/drive/

Moved CVB_poisson_0390_frame2.txt to /content/drive/Shareddrives/Kickstar_team3/dataset/label/val/
Moved CVB_salt_pepper_0396_frame2.txt to /content/drive/Shareddrives/Kickstar_team3/dataset/label/val/
Moved CVB_0405_frame2.txt to /content/drive/Shareddrives/Kickstar_team3/dataset/label/val/
Moved CVB_gaussian_0414_frame2.txt to /content/drive/Shareddrives/Kickstar_team3/dataset/label/val/
Moved CVB_salt_pepper_0414_frame2.txt to /content/drive/Shareddrives/Kickstar_team3/dataset/label/val/
Moved CVB_gaussian_0417_frame2.txt to /content/drive/Shareddrives/Kickstar_team3/dataset/label/val/
Moved CVB_salt_pepper_0420_frame2.txt to /content/drive/Shareddrives/Kickstar_team3/dataset/label/val/
Label file for CVB_salt_pepper_0426_frame2.jpg not found.
Label file for CVB_gaussian_0429_frame2.jpg not found.
Label file for CVB_salt_pepper_0435_frame2.jpg not found.
Moved CVB_gaussian_0441_frame2.txt to /content/drive/Shareddrives/Kickstar_team3/dataset/label/val/
Moved CVB_salt_pepper_0447_fra

Moved CVB_0068_frame1.txt to /content/drive/Shareddrives/Kickstar_team3/dataset/label/val/
Moved CVB_salt_pepper_0068_frame1.txt to /content/drive/Shareddrives/Kickstar_team3/dataset/label/val/
Moved CVB_0069_frame1.txt to /content/drive/Shareddrives/Kickstar_team3/dataset/label/val/
Moved CVB_poisson_0071_frame1.txt to /content/drive/Shareddrives/Kickstar_team3/dataset/label/val/
Moved CVB_poisson_0073_frame1.txt to /content/drive/Shareddrives/Kickstar_team3/dataset/label/val/
Moved CVB_0077_frame1.txt to /content/drive/Shareddrives/Kickstar_team3/dataset/label/val/
Moved CVB_poisson_0077_frame1.txt to /content/drive/Shareddrives/Kickstar_team3/dataset/label/val/
Moved CVB_salt_pepper_0078_frame1.txt to /content/drive/Shareddrives/Kickstar_team3/dataset/label/val/
Moved CVB_poisson_0082_frame1.txt to /content/drive/Shareddrives/Kickstar_team3/dataset/label/val/
Moved CVB_0084_frame1.txt to /content/drive/Shareddrives/Kickstar_team3/dataset/label/val/
Moved CVB_gaussian_0086_frame1.txt

In [ ]:
import os
import logging

# Paths
train_image_dir = '/content/drive/Shareddrives/Kickstar_team3/dataset/images/train'
val_image_dir = '/content/drive/Shareddrives/Kickstar_team3/dataset/images/val'
train_label_dir = '/content/drive/Shareddrives/Kickstar_team3/dataset/label/train'
val_label_dir = '/content/drive/Shareddrives/Kickstar_team3/dataset/label/val'

# Logging setup
logging.basicConfig(filename='label_check_log.txt', level=logging.INFO, format='%(asctime)s - %(message)s')

def check_labels(image_dir, label_dir, subset_name):
    print(f"\n--- Checking {subset_name.upper()} set ---")
    unmatched = []
    matched = 0
    total = 0

    for image_file in os.listdir(image_dir):
        if image_file.endswith('.jpg') or image_file.endswith('.png'):
            total += 1
            image_name = os.path.splitext(image_file)[0]
            label_file = image_name + '.txt'
            label_path = os.path.join(label_dir, label_file)

            if os.path.exists(label_path):
                matched += 1
            else:
                unmatched.append(image_file)
                logging.warning(f"No label found for: {subset_name}/{image_file}")

    unmatched_count = total - matched

    # Summary print
    print(f"Total images in {subset_name}: {total}")
    print(f"✓ Matched labels: {matched}")
    print(f"✗ Unmatched images: {unmatched_count}")

    if unmatched:
        print("Missing labels for the following images:")
        for file in unmatched:
            print("  -", file)

# Run checks
check_labels(train_image_dir, train_label_dir, 'train')
check_labels(val_image_dir, val_label_dir, 'val')

print("\n✅ Label check completed. See 'label_check_log.txt' for unmatched entries.")



--- Checking TRAIN set ---
Total images in train: 8746
✓ Matched labels: 8746
✗ Unmatched images: 0

--- Checking VAL set ---
Total images in val: 980
✓ Matched labels: 980
✗ Unmatched images: 0

✅ Label check completed. See 'label_check_log.txt' for unmatched entries.


In [ ]:
import os
import shutil
import logging

# Paths
train_image_dir = '/content/drive/Shareddrives/Kickstar_team3/dataset/images/train'
val_image_dir = '/content/drive/Shareddrives/Kickstar_team3/dataset/images/val'
train_label_dir = '/content/drive/Shareddrives/Kickstar_team3/dataset/label/train'
val_label_dir = '/content/drive/Shareddrives/Kickstar_team3/dataset/label/val'
junk_dir = '/content/drive/Shareddrives/Kickstar_team3/junk'

# Create junk folder if it doesn't exist
os.makedirs(junk_dir, exist_ok=True)

# Logging setup
logging.basicConfig(filename='label_check_log.txt', level=logging.INFO, format='%(asctime)s - %(message)s')

def move_unmatched_images(image_dir, label_dir, subset_name):
    print(f"\n--- Processing {subset_name.upper()} set ---")
    unmatched = []
    matched = 0
    total = 0

    for image_file in os.listdir(image_dir):
        if image_file.endswith('.jpg') or image_file.endswith('.png'):
            total += 1
            image_name = os.path.splitext(image_file)[0]
            label_file = image_name + '.txt'
            label_path = os.path.join(label_dir, label_file)

            if os.path.exists(label_path):
                matched += 1
            else:
                # Move unmatched image to junk
                source_path = os.path.join(image_dir, image_file)
                target_path = os.path.join(junk_dir, image_file)
                shutil.move(source_path, target_path)
                unmatched.append(image_file)
                logging.warning(f"Moved unmatched image to junk: {subset_name}/{image_file}")
                print(f"Moved: {image_file}")

    print(f"✓ Matched: {matched}")
    print(f"✗ Moved to junk: {len(unmatched)}")
    return unmatched

# Run for train and val
train_unmatched = move_unmatched_images(train_image_dir, train_label_dir, 'train')
val_unmatched = move_unmatched_images(val_image_dir, val_label_dir, 'val')

print(f"\n✅ Done. Total moved to junk: {len(train_unmatched) + len(val_unmatched)}")



--- Processing TRAIN set ---


Moved: Macquarrie_hall_salt_pepper_1308_frame1_train.jpg
Moved: Macquarrie_hall_salt_pepper_1860_frame3_train.jpg
Moved: Macquarrie_hall_poisson_2172_frame3_train.jpg
Moved: CVB_poisson_0097_frame1_train.jpg
Moved: Victory_Salute_gaussian_1221_train.jpg
Moved: Victory_Salute_0267_train.jpg
Moved: Macquarrie_hall_0744_frame3_train.jpg
Moved: Macquarrie_hall_poisson_3048_frame3_train.jpg
Moved: CVB_gaussian_0154_frame1_train.jpg
Moved: Macquarrie_hall_poisson_1272_frame3_train.jpg
Moved: Ceasar_Chavez_Memorial_poisson_0177_train.jpg
Moved: Macquarrie_hall_poisson_4104_frame3_train.jpg
Moved: Macquarrie_hall_1116_frame1_train.jpg
Moved: Ceasar_Chavez_Memorial_2964_train.jpg
Moved: CVB_gaussian_0087_frame1_train.jpg
Moved: Macquarrie_hall_poisson_0192_frame3_train.jpg
Moved: Ceasar_Chavez_Memorial_0681_train.jpg
Moved: Ceasar_Chavez_Memorial_gaussian_0480_train.jpg
Moved: Victory_Salute_gaussian_1425_train.jpg
Moved: Ceasar_Chavez_Memorial_gaussian_1548_train.jpg
Moved: Ceasar_Chavez_Memor

Moved: Ceasar_Chavez_Memorial_gaussian_0963_train.jpg
Moved: Ceasar_Chavez_Memorial_poisson_3273_train.jpg
Moved: Macquarrie_hall_gaussian_4080_frame3_train.jpg
Moved: Ceasar_Chavez_Memorial_gaussian_2640_train.jpg
Moved: Ceasar_Chavez_Memorial_1884_train.jpg
Moved: Ceasar_Chavez_Memorial_1296_train.jpg
Moved: Macquarrie_hall_gaussian_3840_frame3_train.jpg
Moved: CVB_0429_frame2_train.jpg
Moved: Macquarrie_hall_salt_pepper_0396_frame1_train.jpg
Moved: CVB_poisson_0303_frame2_train.jpg
Moved: Ceasar_Chavez_Memorial_gaussian_2565_train.jpg
Moved: Macquarrie_hall_0396_frame1_train.jpg
Moved: CVB_gaussian_0229_frame1_train.jpg
Moved: Victory_Salute_gaussian_1641_train.jpg
Moved: CVB_poisson_0033_frame2_train.jpg
Moved: Ceasar_Chavez_Memorial_gaussian_0387_train.jpg
Moved: Ceasar_Chavez_Memorial_poisson_2973_train.jpg
Moved: Macquarrie_hall_salt_pepper_4020_frame3_train.jpg
Moved: Victory_Salute_gaussian_0615_train.jpg
Moved: Ceasar_Chavez_Memorial_salt_pepper_3306_train.jpg
Moved: Macquarr

Moved: Victory_Salute_salt_pepper_0564.jpg
Moved: Victory_Salute_salt_pepper_0582.jpg
Moved: Victory_Salute_salt_pepper_0618.jpg
Moved: Victory_Salute_salt_pepper_0624.jpg
Moved: Victory_Salute_salt_pepper_0630.jpg
Moved: Victory_Salute_0876.jpg
Moved: Victory_Salute_poisson_0876.jpg
Moved: Victory_Salute_poisson_0885.jpg
Moved: Victory_Salute_0888.jpg
Moved: Victory_Salute_poisson_0888.jpg
Moved: Victory_Salute_poisson_0891.jpg
Moved: Victory_Salute_0894.jpg
Moved: Victory_Salute_poisson_0897.jpg
Moved: Victory_Salute_0900.jpg


Moved: Victory_Salute_0912.jpg
Moved: Victory_Salute_0915.jpg
Moved: Victory_Salute_poisson_0915.jpg
Moved: Victory_Salute_0921.jpg
Moved: Victory_Salute_0942.jpg
Moved: Victory_Salute_0945.jpg
Moved: Victory_Salute_poisson_0945.jpg
Moved: Victory_Salute_poisson_0951.jpg
Moved: Victory_Salute_poisson_0954.jpg
Moved: Victory_Salute_0957.jpg
Moved: Victory_Salute_poisson_0960.jpg
Moved: Victory_Salute_poisson_0963.jpg
Moved: Victory_Salute_0966.jpg
Moved: Victory_Salute_poisson_0966.jpg
Moved: Victory_Salute_0969.jpg
Moved: Victory_Salute_0972.jpg
Moved: Victory_Salute_poisson_1008.jpg
Moved: Victory_Salute_1131.jpg
Moved: Ceasar_Chavez_Memorial_gaussian_0927.jpg
Moved: Ceasar_Chavez_Memorial_salt_pepper_0927.jpg
Moved: Ceasar_Chavez_Memorial_poisson_0927.jpg


...

Moved: Ceasar_Chavez_Memorial_poisson_2037.jpg
Moved: Ceasar_Chavez_Memorial_poisson_2049.jpg
Moved: Ceasar_Chavez_Memorial_salt_pepper_2055.jpg
Moved: Ceasar_Chavez_Memorial_2073.jpg
Moved: Ceasar_Chavez_Memorial_2085.jpg
Moved: Ceasar_Chavez_Memorial_salt_pepper_2085.jpg
Moved: Ceasar_Chavez_Memorial_2103.jpg
Moved: Ceasar_Chavez_Memorial_salt_pepper_2121.jpg
Moved: Ceasar_Chavez_Memorial_gaussian_2133.jpg
Moved: Ceasar_Chavez_Memorial_gaussian_2187.jpg
Moved: Ceasar_Chavez_Memorial_salt_pepper_2187.jpg
Moved: Ceasar_Chavez_Memorial_salt_pepper_2223.jpg
Moved: Ceasar_Chavez_Memorial_2247.jpg
Moved: Ceasar_Chavez_Memorial_poisson_2247.jpg
Moved: Ceasar_Chavez_Memorial_gaussian_2265.jpg
Moved: Ceasar_Chavez_Memorial_salt_pepper_2271.jpg
Moved: Ceasar_Chavez_Memorial_gaussian_2277.jpg
Moved: Ceasar_Chavez_Memorial_gaussian_2301.jpg
Moved: Ceasar_Chavez_Memorial_gaussian_2319.jpg
Moved: Ceasar_Chavez_Memorial_gaussian_2325.jpg
Moved: Ceasar_Chavez_Memorial_salt_pepper_2355.jpg
Moved: Cea

Moved: Ceasar_Chavez_Memorial_2499.jpg
Moved: Ceasar_Chavez_Memorial_salt_pepper_2505.jpg
Moved: Ceasar_Chavez_Memorial_poisson_2505.jpg
Moved: Ceasar_Chavez_Memorial_2535.jpg
Moved: Ceasar_Chavez_Memorial_gaussian_2559.jpg
Moved: Ceasar_Chavez_Memorial_poisson_2559.jpg
Moved: Ceasar_Chavez_Memorial_salt_pepper_2577.jpg
Moved: Ceasar_Chavez_Memorial_2583.jpg
Moved: Ceasar_Chavez_Memorial_gaussian_2583.jpg
Moved: Ceasar_Chavez_Memorial_gaussian_2589.jpg
Moved: Ceasar_Chavez_Memorial_2595.jpg
Moved: Ceasar_Chavez_Memorial_gaussian_2643.jpg
Moved: Ceasar_Chavez_Memorial_salt_pepper_2649.jpg
Moved: Ceasar_Chavez_Memorial_poisson_2649.jpg
Moved: Ceasar_Chavez_Memorial_poisson_2667.jpg
Moved: Ceasar_Chavez_Memorial_0411.jpg
Moved: Ceasar_Chavez_Memorial_salt_pepper_0411.jpg
Moved: Ceasar_Chavez_Memorial_salt_pepper_0423.jpg
Moved: Ceasar_Chavez_Memorial_gaussian_0429.jpg
Moved: Ceasar_Chavez_Memorial_salt_pepper_0429.jpg
Moved: Ceasar_Chavez_Memorial_poisson_0429.jpg
Moved: Ceasar_Chavez_Mem

Moved: Ceasar_Chavez_Memorial_poisson_0675.jpg
Moved: Ceasar_Chavez_Memorial_salt_pepper_0681.jpg
Moved: Ceasar_Chavez_Memorial_0711.jpg
Moved: Ceasar_Chavez_Memorial_gaussian_0711.jpg
Moved: Ceasar_Chavez_Memorial_salt_pepper_0741.jpg
Moved: Ceasar_Chavez_Memorial_0765.jpg
Moved: Ceasar_Chavez_Memorial_poisson_0765.jpg
Moved: Ceasar_Chavez_Memorial_salt_pepper_0783.jpg
Moved: Ceasar_Chavez_Memorial_0807.jpg
Moved: Ceasar_Chavez_Memorial_poisson_0813.jpg
Moved: Ceasar_Chavez_Memorial_0819.jpg
Moved: Ceasar_Chavez_Memorial_gaussian_0843.jpg
Moved: Ceasar_Chavez_Memorial_poisson_0849.jpg
Moved: Ceasar_Chavez_Memorial_0903.jpg
Moved: Ceasar_Chavez_Memorial_poisson_0909.jpg
Moved: Ceasar_Chavez_Memorial_0921.jpg
Moved: Ceasar_Chavez_Memorial_poisson_0933.jpg
Moved: Ceasar_Chavez_Memorial_poisson_0957.jpg
Moved: Ceasar_Chavez_Memorial_0975.jpg
Moved: Ceasar_Chavez_Memorial_0993.jpg
Moved: Ceasar_Chavez_Memorial_gaussian_0993.jpg
Moved: Ceasar_Chavez_Memorial_salt_pepper_1005.jpg
Moved: Ceas

Moved: Ceasar_Chavez_Memorial_1131.jpg
Moved: Ceasar_Chavez_Memorial_salt_pepper_1131.jpg
Moved: Ceasar_Chavez_Memorial_salt_pepper_1167.jpg
Moved: Ceasar_Chavez_Memorial_poisson_1167.jpg
Moved: Victory_Salute_salt_pepper_1383.jpg


Moved: CVB_0303_frame2.jpg
Moved: CVB_0321_frame2.jpg
Moved: CVB_salt_pepper_0426_frame2.jpg
Moved: CVB_gaussian_0429_frame2.jpg
Moved: CVB_salt_pepper_0435_frame2.jpg
Moved: CVB_gaussian_0492_frame2.jpg
Moved: CVB_poisson_0495_frame2.jpg
Moved: CVB_0498_frame2.jpg
Moved: CVB_salt_pepper_0516_frame2.jpg
Moved: CVB_0522_frame2.jpg
Moved: CVB_salt_pepper_0537_frame2.jpg
Moved: CVB_gaussian_0540_frame2.jpg
Moved: CVB_gaussian_0573_frame2.jpg
Moved: CVB_poisson_0576_frame2.jpg
Moved: CVB_0585_frame2.jpg
Moved: CVB_poisson_0591_frame2.jpg
Moved: CVB_0594_frame2.jpg
Moved: CVB_poisson_0594_frame2.jpg
Moved: CVB_salt_pepper_0109_frame1.jpg
✓ Matched: 980
✗ Moved to junk: 248

✅ Done. Total moved to junk: 2613
